In [1]:
import matplotlib.pyplot as plt

In [3]:
import json
import pandas as pd
from collections import defaultdict
import time

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def process_output_file(output_file, microdata_df, debug_mode=True, debug_lines=10):
    # Initialize data structures
    syntpop_stats = defaultdict(lambda: {
        'found': 0,
        'not_found': 0,
        'totals': defaultdict(float)
    })
    
    # Get numeric columns from microdata (excluding ID)
    numeric_cols = [col for col in microdata_df.columns 
                   if col != 'id' and pd.api.types.is_numeric_dtype(microdata_df[col])]
    
    # Create a set of microdata IDs for faster lookup
    microdata_ids = set(microdata_df['id'].values)
    
    start_time = time.time()
    total_rows = 0
    
    with open(output_file, 'r') as file:
        # Skip header
        next(file)
        
        for line in file:
            try:
                total_rows += 1
                parts = line.strip().split(',')
                if len(parts) < 2:
                    continue
                    
                syntpop, census = parts[0], parts[1]
                
                # Try to convert census to int
                try:
                    census_id = int(census)
                except ValueError:
                    syntpop_stats[syntpop]['not_found'] += 1
                    continue
                
                # Check if ID exists
                if census_id in microdata_ids:
                    # Get the matching row (more efficient than querying each time)
                    match_row = microdata_df[microdata_df['id'] == census_id].iloc[0]
                    syntpop_stats[syntpop]['found'] += 1
                    
                    # Update totals for this syntpop
                    for col in numeric_cols:
                        syntpop_stats[syntpop]['totals'][col] += match_row[col]
                else:
                    syntpop_stats[syntpop]['not_found'] += 1
                
                # Debug printing
                if debug_mode and total_rows <= debug_lines:
                    status = "FOUND" if census_id in microdata_ids else "NOT FOUND"
                    print(f"Row {total_rows}: {status} | SyntPop: {syntpop} | CensusID: {census_id}")
                
                # Progress reporting
                if total_rows % 100000 == 0:
                    elapsed = time.time() - start_time
                    print(f"Processed {total_rows:,} rows ({elapsed:.2f} sec)")
                    
                # Early exit for debugging
                if debug_mode and total_rows >= debug_lines:
                    break
                    
            except Exception as e:
                print(f"Error processing line {total_rows}: {str(e)}")
                continue
    
    # Convert to DataFrame
    stats_list = []
    for syntpop, stats in syntpop_stats.items():
        row = {
            'syntpop': syntpop,
            'found': stats['found'],
            'not_found': stats['not_found']
        }
        row.update(stats['totals'])
        stats_list.append(row)
    
    result_df = pd.DataFrame(stats_list)
    
    # Add summary row
    summary_row = {
        'syntpop': 'TOTAL',
        'found': result_df['found'].sum(),
        'not_found': result_df['not_found'].sum()
    }
    for col in numeric_cols:
        summary_row[col] = result_df[col].sum()
    
    result_df = pd.concat([result_df, pd.DataFrame([summary_row])], ignore_index=True)
    
    print(f"\nProcessing complete. Analyzed {total_rows:,} total rows.")
    print(f"Time elapsed: {time.time() - start_time:.2f} seconds")
    
    return result_df


# Read config
config = read_json_file../config.json')

# Load data
microdata_df = pd.read_csv(config['microdata']['file'])

# Process output file
result_df = process_output_file(
    output_file=config['output']['file'],
    microdata_df=microdata_df,
    debug_mode=False,
    debug_lines=10
)

# Display results
print("\nFinal Results:")
print(result_df.head(10))  # Show first 10 syntpop groups
print("\n...")
print(result_df.tail(2))   # Show last group and totals

# Save results to CSV
result_df.to_csv('syntpop_statistics.csv', index=False)
print("\nResults saved to 'syntpop_statistics.csv'")

SyntaxError: unterminated string literal (detected at line 109) (2579748557.py, line 109)

In [3]:
result_df.head(10)

,syntpop,found,not_found,s1_hh_urban_rural%urban,s1_hh_urban_rural%rural,s2_hh_size%hhsize_1,s2_hh_size%hhsize_2,s2_hh_size%hhsize_3,s2_hh_size%hhsize_4,s2_hh_size%hhsize_5,...,s12_employment_hh_size%employed_0_hh_size_4,s12_employment_hh_size%employed_1_hh_size_4,s12_employment_hh_size%employed_2_hh_size_4,s12_employment_hh_size%employed_3_hh_size_4,s13_unpaid_carer_hh_size%carer_0_hh_size_1_2,s13_unpaid_carer_hh_size%carer_0_hh_size_3,s13_unpaid_carer_hh_size%carer_1_hh_size_1_2,s13_unpaid_carer_hh_size%carer_1_hh_size_3,s13_unpaid_carer_hh_size%carer_2_hh_size_1_2,s13_unpaid_carer_hh_size%carer_2_hh_size_3
0,S00135309,71,0,0.0,71.0,26.0,31.0,8.0,4.0,0.0,...,2.0,0.0,1.0,3.0,53.0,11.0,3.0,2.0,1.0,1.0
1,S00135312,64,0,64.0,0.0,12.0,16.0,15.0,14.0,3.0,...,4.0,4.0,11.0,2.0,26.0,24.0,2.0,6.0,0.0,6.0
2,S00135321,45,0,45.0,0.0,20.0,17.0,5.0,0.0,3.0,...,0.0,0.0,3.0,0.0,30.0,8.0,7.0,0.0,0.0,0.0
3,S00135318,42,0,42.0,0.0,16.0,16.0,3.0,5.0,2.0,...,0.0,2.0,5.0,0.0,24.0,8.0,6.0,0.0,2.0,2.0
4,S00135308,33,0,0.0,33.0,7.0,17.0,3.0,4.0,0.0,...,1.0,2.0,3.0,0.0,21.0,5.0,2.0,2.0,1.0,2.0
5,S00135320,59,0,59.0,0.0,20.0,22.0,8.0,8.0,1.0,...,2.0,3.0,3.0,1.0,32.0,13.0,8.0,3.0,2.0,1.0
6,S00135332,47,0,47.0,0.0,21.0,15.0,5.0,3.0,2.0,...,0.0,3.0,1.0,2.0,30.0,9.0,6.0,1.0,0.0,1.0
7,S00135335,48,0,48.0,0.0,11.0,14.0,10.0,9.0,4.0,...,1.0,2.0,7.0,3.0,20.0,16.0,4.0,5.0,1.0,2.0
8,S00135331,31,0,31.0,0.0,10.0,11.0,6.0,2.0,2.0,...,0.0,2.0,2.0,0.0,16.0,5.0,3.0,3.0,2.0,2.0
9,S00135327,43,0,43.0,0.0,13.0,15.0,7.0,5.0,3.0,...,0.0,2.0,4.0,2.0,25.0,10.0,3.0,3.0,0.0,2.0


In [4]:
microdata_df.head()

,id,s1_hh_urban_rural%urban,s1_hh_urban_rural%rural,s2_hh_size%hhsize_1,s2_hh_size%hhsize_2,s2_hh_size%hhsize_3,s2_hh_size%hhsize_4,s2_hh_size%hhsize_5,s2_hh_size%hhsize_6,s2_hh_size%hhsize_7,...,s12_employment_hh_size%employed_0_hh_size_4,s12_employment_hh_size%employed_1_hh_size_4,s12_employment_hh_size%employed_2_hh_size_4,s12_employment_hh_size%employed_3_hh_size_4,s13_unpaid_carer_hh_size%carer_0_hh_size_1_2,s13_unpaid_carer_hh_size%carer_0_hh_size_3,s13_unpaid_carer_hh_size%carer_1_hh_size_1_2,s13_unpaid_carer_hh_size%carer_1_hh_size_3,s13_unpaid_carer_hh_size%carer_2_hh_size_1_2,s13_unpaid_carer_hh_size%carer_2_hh_size_3
0,68006826,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,68013626,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,68020426,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,68027226,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,68047626,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
